# 3. 감성 분류

In [1]:
# 1. 모듈 불러오기
import pandas as pd
from datasets import load_dataset
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TextClassificationPipeline

In [2]:
# 모델 설정 : huggingface에 있는 KOTE 모델 사용하기
# 토크나이저 설정 
tokenizer = AutoTokenizer.from_pretrained("searle-j/kote_for_easygoing_people")
model = AutoModelForSequenceClassification.from_pretrained("searle-j/kote_for_easygoing_people")

# 감성 분석 모델 설정 
pipe = TextClassificationPipeline(
        model=model,
        tokenizer=tokenizer,
        device=0, # gpu number, -1 if cpu used
        return_all_scores=True,
        function_to_apply='sigmoid'
    )

Device set to use cuda:0
/home/wanted-1/miniconda3/envs/team5/lib/python3.11/site-packages/transformers/pipelines/text_classification.py:106: UserWarning: `return_all_scores` is now deprecated,  if want a similar functionality use `top_k=None` instead of `return_all_scores=True` or `top_k=1` instead of `return_all_scores=False`.
  warnings.warn(


In [4]:
df = pd.read_csv(r"/home/wanted-1/potenup-workspace/Project/project2/team5/1.데이터모음/'music_data(Merge)'.csv")
df

,Unnamed: 0,title,artist,lyrics,release_date,likes
0,0,#0000FF,Siera,"Welcome to my world\n\nRed , Blue , Green\n\nM...",2024.12.31,0
1,1,"#RETURN(feat. 장건, Dysearth)",CYAN,how are you doing\n있잖아 그건 나를 falling\nI'm fall...,2024.10.16,23
2,2,#첫사랑,볼빨간사춘기,어릴 때 넌 키도 작고\n빼빼 말랐지 기억나\n난 knock 그런 니가\n괜히 맘에...,2018.01.10,129991
3,3,& The Baby Girl (Feat. SABINE) (Original Mix),George D.Blue,You and I and the baby girl\nWhat should we be...,2024.12.15,50
4,4,'u' (Feat. Vann),Wuno,baby 너만을 바라봐 나는 all day\n알려줘 너만의 private한 stor...,2024.08.26,21
...,...,...,...,...,...,...
4835,4835,흘러가자,유수이 (yusuii),흘러가자 같이 내 시간은 버려두고\n같은 곳을 보면서 share time\n흘러가자...,2024.11.13,10
4836,4836,희망의 빛,데이먼스 이어 (Damons year),피로했던 발을 쉬게 할 수 없던 날\n가망 없다는 걸 처음부터 알고 있었어\n눈꺼풀...,2024.11.19,2541
4837,4837,힌트를 줘요 (Feat. 소라 (SORV)),aib (아입),이렇게 긴 시간이 걸릴 이유가 대체 뭘까\n혼자 있기엔 저 날씨가 아깝달까\n\n만...,2024.11.04,3
4838,4838,힘든 거 알아,PATEKO (파테코),아무 말이라도\n네게 주고 싶은데\n너에게 딱 알맞는 말이 없네\n사람아\n이젠 괜...,2022.12.29,11506


In [5]:
# 데이터 로드
text = df['lyrics'].tolist()

# 긴 텍스트 자르기 함수
def truncate_texts(texts, tokenizer, max_length):   # 입력 텍스트 리스트를 받아, 각 텍스트를 최대 길이로 자르고 잘라낸 텍스트를 반환합니다.
    truncated_texts = []
    for t in texts:
        tokens = tokenizer(t, max_length=max_length, truncation=True)       #텍스트 t를 토큰화할 때, 최대 길이 max_length로 잘라내고 필요하면 truncation=True로 설정하여 초과 부분을 제거합니다.
        truncated_texts.append(tokenizer.decode(tokens['input_ids'], skip_special_tokens=True))     #잘라낸 토큰을 다시 텍스트로 변환하며, 특수 토큰은 제거합니다.
    return truncated_texts

# 텍스트 리스트를 잘라서 파이프라인에 전달
max_length = tokenizer.model_max_length  # 일반적으로 512
truncated_texts = truncate_texts(text, tokenizer, max_length)   # text 리스트의 각 텍스트를 최대 길이로 잘라냅니다.
pip_text = pipe(truncated_texts)        #잘라낸 텍스트를 파이프라인에 전달하여 감성 분석을 수행합니다.


# 결과 저장을 위한 리스트
results = []

# 각 감성에 해당할 확률(score)이 0.4 이상인 감성들만 저장
for i, t in enumerate(truncated_texts):            # 잘라낸 텍스트와 그에 대한 감성 분석 결과를 반복합니다.
    for output in pip_text[i]:
        if output["score"] > 0.4:
            results.append({
                "text": t,
                "label": output["label"],
                "score": output["score"]
            })

# 결과를 데이터프레임으로 변환
results_df = pd.DataFrame(results)

# 데이터프레임 저장 
results_df.to_csv("kote_result.csv", index=False)

# # 각 감성에 해당할 확률(score)이 0.4 이상인 감성들만 출력
# for i, t in enumerate(truncated_texts):     # 잘라낸 텍스트와 그에 대한 감성 분석 결과를 반복합니다.
#     print(f"Text: {t}")
#     for output in pip_text[i]:
#         if output["score"] > 0.4:
#             print(output)
#     print("\n")

ValueError: text input must be of type `str` (single example), `List[str]` (batch or single pretokenized example) or `List[List[str]]` (batch of pretokenized examples).